In [1]:
import sys
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os

sys.path.append("kaggle/input/neurips-open-polymer-prediction-2025")
base_path = "kaggle/input/neurips-open-polymer-prediction-2025/"
supplement_path = "kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/"
tc_smiles_path = "kaggle/input/tc-smiles/"
smiles_extra_data_path = "kaggle/input/smiles-extra-data/"

In [2]:
import torch
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
import optuna
import wandb

from data_preparation import load_qm9_dataset
from train import pretrain_qm9


/scratch/e1350261/venvs/ai39/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
from torch_geometric.datasets import QM9

dataset = QM9(root="data/qm9")
print(dataset)

QM9(130831)


In [4]:
# -----------------------------
# 环境设置
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Training on {device}")

✅ Training on cuda


In [5]:
# -----------------------------
# 数据加载
# -----------------------------
loader, dataset = load_qm9_dataset(root="data/qm9", batch_size=1024)
n = len(dataset)
train_set, val_set = random_split(dataset, [int(0.9*n), n - int(0.9*n)])
train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
val_loader = DataLoader(val_set, batch_size=256)

In [6]:
# -----------------------------
# Optuna 目标函数
# -----------------------------
def objective(trial):
    ckpt_path = f"checkpoints/trial_{trial.number}_best.pt"
    model, hist = pretrain_qm9(
        train_loader=train_loader,
        val_loader=val_loader,
        dataset=dataset,
        hidden_dim=trial.suggest_categorical("hidden_dim", [128, 256, 512]),
        lr=trial.suggest_loguniform("lr", 1e-4, 1e-2),
        num_edge_layers=trial.suggest_int("num_edge_layers", 2, 16),
        dropout=trial.suggest_float("dropout", 0.0, 0.3),
        epochs=20,
        device=device,
        run_wandb=False,
        ckpt_path=ckpt_path
    )
    return min(hist["val_loss"])

In [7]:
import optuna

storage_path = "sqlite:///optuna_qm9.db"  # 保存在当前目录
study = optuna.create_study(
    direction="minimize",
    study_name="qm9_pretrain",
    storage=storage_path,
    load_if_exists=True
)

In [ ]:
# -----------------------------
# 运行超参搜索
# -----------------------------
study.optimize(objective, n_trials=30)

print("✅ 最优超参数:", study.best_trial.params)
print("📉 最优验证集 loss:", study.best_value)

✅ Device: cuda | Params: 19.05M | Tasks: 19


[Epoch 1/20] Train 5039.177, Val 2012.391 | mu:1.049 | alpha:4.661 | homo:0.481 | lumo:0.880 | gap:0.921 | r2:189.494 | zpve:0.476 | U0:435.961 | U:435.010 | H:451.903 | G:468.354 | Cv:2.669 | u0_atom:5.312 | u_atom:4.575 | h_atom:4.816 | g_atom:4.345 | A:0.904 | B:0.339 | C:0.242


[Epoch 2/20] Train 2823.700, Val 1994.852 | mu:1.084 | alpha:5.112 | homo:0.756 | lumo:0.767 | gap:0.908 | r2:190.129 | zpve:0.627 | U0:381.626 | U:501.141 | H:481.400 | G:395.035 | Cv:2.982 | u0_atom:8.802 | u_atom:7.113 | h_atom:8.060 | g_atom:7.843 | A:0.929 | B:0.322 | C:0.218


[Epoch 3/20] Train 2720.030, Val 2239.258 | mu:0.960 | alpha:3.908 | homo:0.528 | lumo:0.663 | gap:0.821 | r2:163.404 | zpve:0.420 | U0:476.142 | U:457.036 | H:503.516 | G:614.772 | Cv:2.144 | u0_atom:3.104 | u_atom:3.520 | h_atom:3.853 | g_atom:3.053 | A:0.897 | B:0.306 | C:0.211


[Epoch 4/20] Train 2710.695, Val 2000.603 | mu:0.952 | alpha:3.827 | homo:0.758 | lumo:0.619 | gap:0.689 | r2:153.537 | zpve:0.312 | U0:433.255 | U:510.246 | H:495.418 | G:379.490 | Cv:2.226 | u0_atom:4.168 | u_atom:3.446 | h_atom:4.578 | g_atom:5.644 | A:0.935 | B:0.303 | C:0.200


...

[Epoch 8/20] Train 2680.722, Val 1712.158 | mu:0.913 | alpha:3.387 | homo:0.574 | lumo:0.610 | gap:0.807 | r2:145.859 | zpve:0.298 | U0:440.979 | U:354.160 | H:374.204 | G:371.557 | Cv:1.769 | u0_atom:5.135 | u_atom:3.722 | h_atom:4.044 | g_atom:2.784 | A:0.882 | B:0.292 | C:0.183


[Epoch 9/20] Train 2674.557, Val 1793.320 | mu:0.957 | alpha:3.742 | homo:0.362 | lumo:0.607 | gap:0.835 | r2:147.047 | zpve:0.179 | U0:417.342 | U:348.891 | H:354.708 | G:501.976 | Cv:2.438 | u0_atom:4.213 | u_atom:3.281 | h_atom:2.844 | g_atom:2.530 | A:0.861 | B:0.312 | C:0.196


[Epoch 10/20] Train 2668.026, Val 2058.296 | mu:0.941 | alpha:3.133 | homo:0.368 | lumo:0.513 | gap:0.648 | r2:157.066 | zpve:0.359 | U0:489.679 | U:495.757 | H:453.833 | G:440.164 | Cv:1.979 | u0_atom:2.816 | u_atom:3.224 | h_atom:3.562 | g_atom:2.933 | A:0.856 | B:0.283 | C:0.184


[W 2025-09-08 01:42:12,732] Trial 0 failed with parameters: {'hidden_dim': 2048, 'num_edge_layers': 5, 'dropout': 0.08739049662913614, 'lr': 0.0033953307338137396} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/scratch/e1350261/venvs/ai39/lib64/python3.9/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/var/tmp/pbs.206459.stdct-mgmt-02/ipykernel_3890535/2582930910.py", line 10, in objective
    _, hist = pretrain_qm9(
  File "/nfs/home/svu/e1350261/kaggle/kaggle-neurips-0907/train.py", line 157, in pretrain_qm9
    history = fit(model, train_loader, val_loader, optimizer, device, tasks,
  File "/nfs/home/svu/e1350261/kaggle/kaggle-neurips-0907/train.py", line 105, in fit
    tr_loss, tr_mae = train_one_epoch(model, train_loader, optimizer, device, tasks, use_mask, epoch, epochs)
  File "/nfs/home/svu/e1350261/kaggle/kaggle-neurips-0907/train.py", line 25, in train_one_epoch
    

KeyboardInterrupt: 